In [0]:
%run /Workspace/Users/loganathansundaramsa@gmail.com/dataengg_inc_learning/databricks_learning/Brazilian_Ecommerce/gendral_config/com_utl_fun

In [0]:
import json

gendral_config = dbutils.notebook.run("/Workspace/Users/loganathansundaramsa@gmail.com/dataengg_inc_learning/databricks_learning/Brazilian_Ecommerce/gendral_config/common_notebook", 120)


config_dict = json.loads(gendral_config)

src = config_dict['src']
bronze = config_dict['bronze']
silver = config_dict['silver']
gold = config_dict['gold']
catalog = config_dict['CATALOG']
schema = config_dict['SCHEMA']

In [0]:
df_orders = read_file(spark,'parquet',f'{bronze}/orders')


In [0]:
orders_schema = {
    "order_purchase_timestamp": "timestamp",
    "order_delivered_customer_date": "timestamp",
    "order_estimated_delivery_date": "timestamp",
    "order_delivered_carrier_date": "timestamp",
    "order_approved_at": "timestamp"
}

df = f_cast_col(df_orders,orders_schema)

In [0]:
df_rmd = deduplicate(df,["order_id"])
df_dropnull = drop_required_nulls(df_rmd,["order_id","order_purchase_timestamp"])

In [0]:
# print(df_dropnull.count())
df_final_ord = df_dropnull.filter("order_delivered_customer_date >= order_purchase_timestamp") \
    .filter("lower(order_status) IN ('delivered','shipped','canceled','invoiced')") \
    .withColumn("delivery_days",date_diff(col("order_delivered_customer_date"),col("order_purchase_timestamp"))) \
    .withColumn("delay_flag",when(col("order_delivered_customer_date")>col("order_estimated_delivery_date"),1).otherwise(0))

columns = {
        "created_at": {"type": "timestamp"},
        "updated_at": {"type": "timestamp"},
        "source": {"type": "string", "default": "orders"}}

df_final_ord = enrich_add_col(df_final_ord, columns)


# df_final_ord.display()

In [0]:
# Write data delta file /delta table in silver location

write_file(df_final_ord,f"{silver}/silver_orders")
write_delta_liquid(df_final_ord, f"{catalog}.{schema}.silver_orders",["order_id","customer_id"])
